In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics import precision_score, recall_score
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
import torch
from pytorch_lightning import Trainer, LightningModule
from pytorch_lightning.callbacks import ModelCheckpoint, Callback
from pytorch_lightning.loggers import TensorBoardLogger
from sentence_transformers import SentenceTransformer
from torch.utils.data import DataLoader, TensorDataset
import faiss
import numpy as np

D:\Anaconda\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3060 Laptop GPU'

In [3]:
# Load datasets
users = pd.read_csv('processed_dataset/MovieLens-1M/users/users_movielens.csv')
movies = pd.read_csv('processed_dataset/MovieLens-1M/movies/movies_movielens.csv')
train_ratings = pd.read_csv('processed_dataset/MovieLens-1M/ratings/traindata_movielens.csv')
val_ratings = pd.read_csv('processed_dataset/MovieLens-1M/ratings/valdata_movielens.csv')
test_ratings = pd.read_csv('processed_dataset/MovieLens-1M/ratings/testdata_movielens.csv')

In [4]:
sbert_model = SentenceTransformer("all-MiniLM-L6-v2")
# sbert_model = SentenceTransformer("intfloat/e5-small-v2")
# sbert_model = SentenceTransformer("all-mpnet-base-v2")

D:\Anaconda\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
sbert_model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [6]:
train_ratings

,user_id,item_id,rating,timestamp
0,6040,593,5,956703954
1,6040,1961,4,956703977
2,6040,2019,5,956703977
3,6040,1419,3,956704056
4,6040,3505,4,956704056
...,...,...,...,...
600120,403,1036,4,1046388675
600121,5948,1180,5,1046437879
600122,5948,3098,4,1046437932
600123,4958,1407,5,1046454443


In [7]:
# Combine user features into a single string for each user
users['user_features'] = 'occupation: ' + users['occupation'] + ' [SEP] age: ' + users['age'].astype(str) + ' [SEP] gender: ' + users['gender'].astype(str)

# Combine movie features into a single string for each movie
movies['movie_features'] = 'title: ' + movies['title'] + ' [SEP] genres: ' + movies['genres']

In [8]:
# Create a dictionary for fast lookup
user_features_dict = users.set_index('user_id')['user_features'].to_dict()
movie_features_dict = movies.set_index('item_id')['movie_features'].to_dict()

# Create lists of user and item texts
user_texts = [user_features_dict[userId] for userId in train_ratings['user_id'].unique()]
item_texts = [movie_features_dict[movieId] for movieId in train_ratings['item_id'].unique()]

# Create a mapping from userId and movieId to indices
user_id_to_idx = {userId: idx for idx, userId in enumerate(train_ratings['user_id'].unique())}
movie_id_to_idx = {movieId: idx for idx, movieId in enumerate(train_ratings['item_id'].unique())}

# Map userId and movieId in ratings_df to indices
train_ratings['user_idx'] = train_ratings['user_id'].map(user_id_to_idx)
train_ratings['movie_idx'] = train_ratings['item_id'].map(movie_id_to_idx)

# Map userId and movieId in ratings_val to indices
val_ratings['user_idx'] = val_ratings['user_id'].map(user_id_to_idx)
val_ratings['movie_idx'] = val_ratings['item_id'].map(movie_id_to_idx)

# Extract user indices, item indices, and ratings
user_indices = torch.LongTensor(train_ratings['user_idx'].values).to(device)
item_indices = torch.LongTensor(train_ratings['movie_idx'].values).to(device)
labels = torch.FloatTensor(train_ratings['rating'].values).to(device)

# Extract user indices, item indices, and ratings for validation
val_user_indices = torch.LongTensor(val_ratings['user_idx'].values).to(device)
val_item_indices = torch.LongTensor(val_ratings['movie_idx'].values).to(device)
val_labels = torch.FloatTensor(val_ratings['rating'].values).to(device)

In [9]:
user_texts

['occupation: doctor/health care [SEP] age: 25-34 [SEP] gender: Male',
 'occupation: other or not specified [SEP] age: 45-49 [SEP] gender: Female',
 'occupation: academic/educator [SEP] age: 56+ [SEP] gender: Female',
 'occupation: academic/educator [SEP] age: 45-49 [SEP] gender: Female',
 'occupation: scientist [SEP] age: 25-34 [SEP] gender: Female',
 'occupation: academic/educator [SEP] age: 25-34 [SEP] gender: Female',
 'occupation: sales/marketing [SEP] age: 25-34 [SEP] gender: Male',
 'occupation: retired [SEP] age: 50-55 [SEP] gender: Male',
 'occupation: executive/managerial [SEP] age: 45-49 [SEP] gender: Male',
 'occupation: other or not specified [SEP] age: 18-24 [SEP] gender: Female',
 'occupation: technician/engineer [SEP] age: 25-34 [SEP] gender: Male',
 'occupation: academic/educator [SEP] age: 25-34 [SEP] gender: Female',
 'occupation: college/grad student [SEP] age: 18-24 [SEP] gender: Male',
 'occupation: college/grad student [SEP] age: 18-24 [SEP] gender: Male',
 'occu

In [10]:
user_indices

tensor([   0,    0,    0,  ...,   90, 1081, 1081], device='cuda:0')

In [11]:
item_indices

tensor([   0,    1,    2,  ..., 2120,  722, 1234], device='cuda:0')

In [12]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
import torch
from torchmetrics import Precision, Recall
from pytorch_lightning import Trainer, LightningModule

In [13]:
user_embeddings = torch.tensor(sbert_model.encode(user_texts)).to(device)
item_embeddings = torch.tensor(sbert_model.encode(item_texts)).to(device)

D:\Anaconda\lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [14]:
user_indices

tensor([   0,    0,    0,  ...,   90, 1081, 1081], device='cuda:0')

In [15]:
# Create DataLoader for training data
train_dataset = TensorDataset(user_indices, item_indices, labels)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last=True)

# Create DataLoader for validation data
val_dataset = TensorDataset(val_user_indices, val_item_indices, val_labels)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, drop_last=True)

In [16]:
# Assuming you have already defined and initialized your train_dataloader
for batch in train_dataloader:
    user_batch, item_batch, label_batch = batch
    print(f"User indices shape: {user_batch.shape}")
    print(f"Item indices shape: {item_batch.shape}")
    print(f"Labels shape: {label_batch.shape}")
    # Optionally print some actual data samples
    print("Sample user indices:", user_batch[1])
    print("Sample item indices:", item_batch[1])
    print("Sample label:", label_batch[0])
    print(batch)
    # Break after printing the first batch if you just want to see one batch
    break

User indices shape: torch.Size([32])
Item indices shape: torch.Size([32])
Labels shape: torch.Size([32])
Sample user indices: tensor(1853, device='cuda:0')
Sample item indices: tensor(2593, device='cuda:0')
Sample label: tensor(2., device='cuda:0')
[tensor([4765, 1853, 2443, 4917, 5984, 3102, 3732, 4268, 5190, 3377, 2006, 4421,
         645, 3929,  680,  201, 1462,  591, 5852, 3121, 5428, 5020,  252, 2374,
         928, 2508, 1432, 2696, 1344, 4306, 5881, 5289], device='cuda:0'), tensor([1221, 2593,  217, 1966,  452,  874,  933, 2047, 2257,  422,  887,  159,
         573, 1590,  598, 1055,  522,  115,  324, 2935,   17, 1539,  770, 1488,
        2136, 1296, 1186, 3035, 1224,  299, 1158,  400], device='cuda:0'), tensor([2., 3., 3., 2., 3., 5., 4., 5., 5., 2., 5., 2., 2., 5., 4., 4., 4., 2.,
        3., 3., 5., 1., 2., 4., 4., 3., 5., 3., 3., 5., 4., 3.],
       device='cuda:0')]


In [17]:
for batch_idx, batch in enumerate(val_dataloader):
    val_user_batch, val_item_batch, val_label_batch = batch
    assert val_user_batch.shape[0] == 32, f"Unexpected batch size at index {batch_idx}: {val_item_batch.shape[0]}"


In [18]:
val_ratings

,user_id,item_id,rating,timestamp,user_idx,movie_idx
0,6040,858,4,956703932,0,494.0
1,6040,3111,5,956704056,0,442.0
2,6040,213,5,956704056,0,221.0
3,6040,608,4,956704475,0,1050.0
4,6040,1649,5,956704519,0,2288.0
...,...,...,...,...,...,...
200037,4958,2453,4,1046454260,1081,1977.0
200038,4958,2043,1,1046454282,1081,2255.0
200039,4958,2399,1,1046454338,1081,1852.0
200040,4958,2634,3,1046454548,1081,1913.0


In [19]:
# Reverse the user_id_to_idx dictionary to map indices back to user IDs
idx_to_user_id = {idx: userId for userId, idx in user_id_to_idx.items()}

# Get the real user ID for user_idx 1
real_user_id = idx_to_user_id[519]
print(f"Real user ID for user_idx 1: {real_user_id}")
# Get the text features for the identified real user ID
user_text_features = user_features_dict[real_user_id]
print(f"Text features for user ID {real_user_id}: {user_text_features}")

Real user ID for user_idx 1: 5520
Text features for user ID 5520: occupation: academic/educator [SEP] age: 45-49 [SEP] gender: Male


In [20]:
# Reverse the user_id_to_idx dictionary to map indices back to user IDs
idx_to_item_id = {idx: itemId for itemId, idx in movie_id_to_idx.items()}

# Get the real user ID for user_idx 1
real_user_id = idx_to_item_id[1628]
print(f"Real user ID for user_idx 1: {real_user_id}")
# Get the text features for the identified real user ID
user_text_features = movie_features_dict[real_user_id]
print(f"Text features for user ID {real_user_id}: {user_text_features}")

Real user ID for user_idx 1: 2110
Text features for user ID 2110: title: Dead Men Don't Wear Plaid (1982) [SEP] genres: Comedy|Crime|Thriller


In [21]:
# Identify missing entries
missing_item_ids = set(val_ratings['item_id'].unique()) - set(movie_features_dict.keys())
print("Missing item IDs:", missing_item_ids)

Missing item IDs: set()


In [22]:
print(user_embeddings.shape)  # Check the shape of user_embeddings
print(item_embeddings.shape)

torch.Size([6040, 384])
torch.Size([3630, 384])


In [23]:
from torchmetrics import MeanSquaredError, MeanAbsoluteError
from torchmetrics.retrieval import RetrievalPrecision, RetrievalRecall

In [47]:
class TwoTowerModel(LightningModule):
    def __init__(self, embedding_dim, user_texts, item_texts, criterion, learning_rate=0.001):
        super(TwoTowerModel, self).__init__()
        # self.sbert_model = SentenceTransformer("all-MiniLM-L6-v2")
        self.user_embeddings = torch.tensor(sbert_model.encode(user_texts), device=self.device)
        self.item_embeddings = torch.tensor(sbert_model.encode(item_texts), device=self.device)
        self.user_embedding_layer = nn.Embedding.from_pretrained(self.user_embeddings, freeze=False)
        self.item_embedding_layer = nn.Embedding.from_pretrained(self.item_embeddings, freeze=False)
        self.fc = nn.Linear(embedding_dim * 2, 1)
        self.criterion = criterion
        self.learning_rate = learning_rate
        self.epoch_losses = {'train_loss': [], 'val_loss': []}  # Initialize epoch_losses as a dictionary

        # Initialize metrics
        self.mae = MeanAbsoluteError()
        self.precision_at_k = RetrievalPrecision(top_k=10)
        self.recall_at_k = RetrievalRecall(top_k=10)

        # For accumulating predictions and targets
        self.predictions = []
        self.targets = []
        self.indexes = []

        # Initialize dictionaries to store epoch metrics
        self.epoch_losses = {
            'train_loss': [],
            'val_loss': [],
            'val_precision_at_k': [],
            'val_recall_at_k': [],
            'val_mae': [],
        }

    def forward(self, user_indices, item_indices):
        user_embedding = self.user_embedding_layer(user_indices)
        item_embedding = self.item_embedding_layer(item_indices)
        combined_embeddings = torch.cat((user_embedding, item_embedding), dim=1)
        output = self.fc(combined_embeddings).squeeze()
        return torch.clamp(4 * torch.sigmoid(output) + 1, min=1, max=5)

    def training_step(self, batch, batch_idx):
        user_indices, item_indices, labels = batch
        predictions = self.forward(user_indices, item_indices)
        loss = self.criterion(predictions, labels)
        self.log('train_loss', loss, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        user_indices, item_indices, labels = batch
        predictions = self.forward(user_indices, item_indices)
        loss = self.criterion(predictions, labels)

        self.predictions.append(predictions)
        self.targets.append((labels > 3.5).int())  # Example threshold for relevance
        self.indexes.append(user_indices)


        # Calculate additional metrics
        mae = self.mae(predictions, labels)
        self.log('val_loss', loss, on_epoch=True, prog_bar=True)
        self.log('val_mae', mae, on_epoch=True, prog_bar=True)

        return loss

    def on_validation_epoch_end(self):
        # Concatenate all predictions, targets, and indexes
        predictions = torch.cat(self.predictions)
        targets = torch.cat(self.targets)
        indexes = torch.cat(self.indexes)

        # Compute precision and recall at k
        precision_k = self.precision_at_k(predictions, targets, indexes)
        recall_k = self.recall_at_k(predictions, targets, indexes)

        # Log precision and recall
        self.log('val_precision_at_k', precision_k, on_epoch=True, prog_bar=True)
        self.log('val_recall_at_k', recall_k, on_epoch=True, prog_bar=True)

        # Clear lists for next epoch
        self.predictions.clear()
        self.targets.clear()
        self.indexes.clear()

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.learning_rate)

    def get_user_embeddings(self, user_indices):
        return self.user_embedding_layer(user_indices)

    def get_item_embeddings(self, item_indices=None):
        if item_indices is None:
            item_indices = torch.arange(self.item_embedding_layer.num_embeddings).to(self.device)
        return self.item_embedding_layer(item_indices)

In [48]:
# Custom callback to print the losses after each epoch
from pytorch_lightning.callbacks import ModelCheckpoint, Callback

class PrintLossesCallback(Callback):
    def on_train_epoch_end(self, trainer, pl_module):
        train_loss = trainer.callback_metrics['train_loss']
        val_loss = trainer.callback_metrics['val_loss']
        val_precision = trainer.callback_metrics.get('val_precision_at_k')
        val_recall = trainer.callback_metrics.get('val_recall_at_k')
        val_mae = trainer.callback_metrics.get('val_mae')
        epoch = trainer.current_epoch
        pl_module.epoch_losses['train_loss'].append(train_loss.item())
        pl_module.epoch_losses['val_loss'].append(val_loss.item())
        pl_module.epoch_losses['val_precision_at_k'].append(val_precision.item())
        pl_module.epoch_losses['val_recall_at_k'].append(val_recall.item())
        pl_module.epoch_losses['val_mae'].append(val_mae.item())
        print(f"Epoch {epoch + 1}: Train Loss: {train_loss}, Val Loss: {val_loss}")



In [49]:
user_embeddings.size()[1]

384

In [50]:
# Initialize model
embedding_dim = user_embeddings.size()[1] # Adjust based on your sentence transformer model's output size
criterion = nn.MSELoss()
model = TwoTowerModel(embedding_dim, user_texts, item_texts, criterion)

logger = TensorBoardLogger("tb_logs", name="all-MiniLM-L6-v2_5-epochs_lr-0.001_no-history")

# Setup PyTorch Lightning Trainer
checkpoint_callback = ModelCheckpoint(monitor='val_loss', mode='min', save_top_k=1, filename='all-MiniLM-L6-v2_5-epochs_lr-0.001_no-history')

trainer = Trainer(
    max_epochs=5,
    logger=logger,
    callbacks=[checkpoint_callback, PrintLossesCallback()],
    enable_progress_bar=True,
)

# Fit the model
trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                 | Type               | Params | Mode 
--------------------------------------------------------------------
0 | user_embedding_layer | Embedding          | 2.3 M  | train
1 | item_embedding_layer | Embedding          | 1.4 M  | train
2 | fc                   | Linear             | 769    | train
3 | criterion            | MSELoss            | 0      | train
4 | mae                  | MeanAbsoluteError  | 0      | train
5 | precision_at_k       | RetrievalPrecision | 0      | train
6 | recall_at_k          | RetrievalRecall    | 0      | train
--------------------------------------------------------------------
3.7 M     Trainable params
0         Non-trainable params
3.7 M     Total params
14.856    Total estimated model params size (MB)


D:\Anaconda\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
D:\Anaconda\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Epoch 0: 100%|██████████| 18753/18753 [02:01<00:00, 153.89it/s, v_num=1, train_loss_step=0.666]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 1: 100%|██████████| 18753/18753 [01:59<00:00, 157.48it/s, v_num=1, train_loss_step=1.280, val_loss=0.848, val_mae=0.732, val_precision_at_k=0.663, val_recall_at_k=0.623, train_loss_epoch=0.881]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 2: 100%|██████████| 18753/18753 [02:00<00:00, 155.71it/s, v_num=1, train_loss_step=0.430, val_loss=0.837, val_mae=0.723, val_precision_at_k=0.664, val_recall_at_k=0.623, train_loss_epoch=0.841]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 3: 100%|██████████| 18753/18753 [02:01<00:00, 154.91it/s, v_num=1, train_loss_step=0.784, val_loss=0.852, val_mae=0.733, val_precision_at_k=0.665, val_recall_at_k=0.624, train_loss_epoch=0.834]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|██████████| 18753/18753 [02:05<00:00, 149.65it/s, v_num=1, train_loss_step=0.617, val_loss=0.859, val_mae=

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 18753/18753 [02:50<00:00, 110.19it/s, v_num=1, train_loss_step=0.617, val_loss=0.838, val_mae=0.719, val_precision_at_k=0.665, val_recall_at_k=0.624, train_loss_epoch=0.830]


In [16]:
tensorboard --logdir tb_logs

SyntaxError: invalid syntax (3304409492.py, line 1)

In [28]:
# Print losses after training completes
print("Epoch losses:")
for epoch in range(trainer.max_epochs):
    print(f"Epoch {epoch + 1}: Train Loss: {model.epoch_losses['train_loss'][epoch]}, Val Loss: {model.epoch_losses['val_loss'][epoch]}")

Epoch losses:
Epoch 1: Train Loss: 1.3757036924362183, Val Loss: 1.2727571725845337
Epoch 2: Train Loss: 1.2392809391021729, Val Loss: 1.216723918914795
Epoch 3: Train Loss: 1.2070212364196777, Val Loss: 1.1934220790863037
Epoch 4: Train Loss: 1.18389093875885, Val Loss: 1.1698319911956787
Epoch 5: Train Loss: 1.1590394973754883, Val Loss: 1.1441967487335205


In [29]:
# Print losses after training completes
print("Epoch losses:")
for epoch in range(trainer.max_epochs):
    print(f"Epoch {epoch + 1}: Train Loss: {model.epoch_losses['train_loss'][epoch]}, Val Loss: {model.epoch_losses['val_loss'][epoch]}, Val Precision@10: {model.epoch_losses['val_precision_at_k'][epoch]}, Val Recall@10: {model.epoch_losses['val_recall_at_k'][epoch]}, Val MAE: {model.epoch_losses['val_mae'][epoch]}")

Epoch losses:
Epoch 1: Train Loss: 1.3757036924362183, Val Loss: 1.2727571725845337, Val Precision@10: 0.6116653084754944, Val Recall@10: 0.5954939126968384, Val MAE: 0.9583308100700378
Epoch 2: Train Loss: 1.2392809391021729, Val Loss: 1.216723918914795, Val Precision@10: 0.6382104754447937, Val Recall@10: 0.6088075041770935, Val MAE: 0.9271165132522583
Epoch 3: Train Loss: 1.2070212364196777, Val Loss: 1.1934220790863037, Val Precision@10: 0.648980975151062, Val Recall@10: 0.6143777370452881, Val MAE: 0.9128139615058899
Epoch 4: Train Loss: 1.18389093875885, Val Loss: 1.1698319911956787, Val Precision@10: 0.6534879803657532, Val Recall@10: 0.6170315742492676, Val MAE: 0.9015042781829834
Epoch 5: Train Loss: 1.1590394973754883, Val Loss: 1.1441967487335205, Val Precision@10: 0.6563048362731934, Val Recall@10: 0.6186469793319702, Val MAE: 0.8892715573310852


In [1]:
# Load the best model checkpoint
checkpoint_path = "tb_logs/two_tower_model/version_10/checkpoints/e5-two-tower-model.ckpt"
model = TwoTowerModel.load_from_checkpoint(checkpoint_path, embedding_dim=384, user_texts=user_texts, item_texts=item_texts, criterion=nn.MSELoss())
model.eval()

NameError: name 'TwoTowerModel' is not defined

In [30]:
import faiss

def build_faiss_index(item_embeddings):
    d = item_embeddings.shape[1]  # Dimension of the embeddings
    index = faiss.IndexFlatL2(d)  # Create a FAISS index
    index.add(item_embeddings)  # Add item embeddings to the index
    return index

In [44]:
def get_top_n_recommendations(model, user_ids, n=10):
    model.eval()

    # Convert user_ids to tensor indices
    user_indices = torch.tensor([user_id_to_idx[user_id] for user_id in user_ids]).to(model.device)

    # Get user embeddings
    user_embeddings = model.get_user_embeddings(user_indices).detach().cpu().numpy()

    # Get all item embeddings
    item_indices = torch.arange(model.item_embedding_layer.num_embeddings).to(model.device)
    item_embeddings = model.get_item_embeddings(item_indices).detach().cpu().numpy()

    # Build FAISS index
    index = build_faiss_index(item_embeddings)

    # Get top N recommendations for each user
    top_n_recommendations = {}
    D, I = index.search(user_embeddings, n)  # I contains indices of the top N items, D contains distances

    for i, user_id in enumerate(user_ids):
        min_dist = D[i].min()
        max_dist = D[i].max()

        scores = 1 + 4 * (max_dist - D[i]) / (max_dist - min_dist)
        top_n_items_with_scores = pd.Series(data=scores, index=item_indices[I[i]].cpu().numpy())
        top_n_recommendations[user_id] = top_n_items_with_scores

    return top_n_recommendations

In [51]:
# Example usage:
user_ids = test_ratings['user_id'].unique()  # List of user IDs in the test dataset
n = 5 # Number of recommendations per user

recommendations = get_top_n_recommendations(model, user_ids, n)

In [52]:
from evaluator import EvaluateMetrics

# Evaluate the recommendations
evaluator = EvaluateMetrics(test_ratings)
evaluation_results = evaluator.evaluate_recommendations(recommendations, k=n)

print(f"NDCG@{n}: {evaluation_results['NDCG@k']:.4f}")
print(f"MRR@{n}: {evaluation_results['MRR@k']:.4f}")
print(f"HR@{n}: {evaluation_results['HR@k']:.4f}")

NDCG@5: 0.5645
MRR@5: 0.4231
HR@5: 1.0000
